## Objetivo 1: lectura, tratamiento y carga de datos

In [59]:
import zipfile
import re, glob, os, math
import pandas as pd
import numpy as np

### 1.1 Lectura de datos

En este caso los datos se pueden descargar directamente en formato .zip desde la página oficial para poder trabajar con ellos desde manera local.
* (https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=41e01e007c9db410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

En el caso que se encontraran solo dispuestos a través de la url, se haría uso de la librería 'urllib3'. Esta librería te permite conectarte a la página y hacer scraping en los datos.

Tras revisar como están dispuestos los datos, elaboro 3 funciones en el que permita extraer los ficheros en sus diferentes formatos(.txt, .csv, .xml).
Puede ser útil para en un futuro se decide cambiar según su formato al inicial.

Para esta prueba técnica he elegido el formato .csv  para mostrar los diferentes pasos de ETL.

In [60]:
def get_txt_file():
    '''Retrieve file '''
    txt_files=[]
    zip_ref = zipfile.ZipFile('Anio201810.zip','r')
    paths = zip_ref.namelist()
    for path in paths:
        pattern = re.compile(r'\b.txt\b')
        reTXT = pattern.search(path)
        if reTXT is not None:
            file=zip_ref.extract(path)
            txt_files.append(file)    
    return txt_files

def get_csv_file():
    '''Retrieve file '''
    csv_files=[]
    zip_ref = zipfile.ZipFile('Anio201810.zip','r')
    paths = zip_ref.namelist()
    for path in paths:
        pattern = re.compile(r'\b.csv\b')
        reCSV = pattern.search(path)
        if reCSV is not None:
            file=zip_ref.extract(path)
            csv_files.append(file)    
    return csv_files

def get_xml_file():
    '''Retrieve file '''
    xml_files=[]
    zip_ref = zipfile.ZipFile('Anio201810.zip','r')
    paths = zip_ref.namelist()
    for path in paths:
        pattern = re.compile(r'\b.xml\b')
        reXML = pattern.search(path)
        if reXML is not None:
            file=zip_ref.extract(path)
            xml_files.append(file)    
    return xml_files

### 1.2. Lectura y tratamiento de datos
Para este apartado se va a proceder a la carga de los ficheros .csv a un dataframe, en el que se pretende hacer los siguientes pasos por orden:

* Lectura de los ficheros .csv a dataframe
* Concatenación de los dataframes
* Ordenar de manera ascendente según el mes y día de mes
* Revisión que no se pierdan datos durante el proceso

Es interesante para este caso ordenar los datos según el mes y día de manera ascendete para posteriores análisis y represantciones gráficas y así cononocer la evolución de las concentraciones de NO2

Se divide el proceso en dos funciones diferentes:

* Función donde se realiza la lectura, concatenación y ordenar los datos (processing_data)
* Función para comprobar que durante el proceso no se hayan perdido datos en el camino (check_processing_data)


In [103]:
def processing_data():
    '''Concatenate dataframe and sort'''
    
    df1=pd.DataFrame()
    csv_file=get_csv_file()
    for path in csv_file:
        df = pd.read_csv(path, delimiter=";")
        if not df1.empty:
            df=pd.concat([df, df1])
        df1=df
    df = df1.sort_values(['MES','DIA'], ascending=(True,True))   
    return df

Es importante comprobar que se haya hecho la carga de los ficheros en dataframe de manera satisfactoria y no se haya perdido alguno en el camino.
Para ello elaboro una función donde compruebe la suma de las filas por cada fichero sea igual al 'df'.

In [102]:
def check_processing_data():
    
    array_rows=[]
    csv_file=get_csv_file()
    total_df_rows=processing_data()
    total_df_rows=total_df_rows.shape[0]
    '''Sum row files'''
    for path in csv_file:
        dataframe = pd.read_csv(path, delimiter=";")
        rows=dataframe.shape[0]
        array_rows=array_rows+[rows]
    total_array_rows=np.sum(array_rows)
    '''Compare row numbers df and row files'''
    if total_df_rows==total_array_rows:
        print('Satisfactory process')
    else:
        print('Something went wrong')
    
check_processing_data()   

Satisfactory process


### 1.3 Data Wrangling: Tratamiento de los datos

En este apartado se pretende hacer un tratamiento de datos para tener solo aquellos que nos aporte valor a la hora de realizar análisis de NO2 y representaciones gráficas.

Para ello este apartado se divide en dos partes:

1. Filtrado de datos NO2 y valores válidos
2. Filtrado de valores Null y valores Nan

A su vez estará compuesto por 3 funciones:

* Función de filtrado NO2 y valores válidos (filtered_data)
* Función filtrado valores Null
* Función filtrado valores Nan

Funcion específica de python (pd.isnull), que devuelve un vector en colummnas con valores True/False, donde True indica que es null, False no es null

Para este ejercicio haré uso de la función pd.isnull/pd.isnan en el filtrado de valores Null y valores Nan

Nota: La existencia de valores null se puede dar por varias razones; a la hora de extraer los datos de la base de datos puede haber incompatibilidades y en la recolección de los datos antes de guardarse en la base de datos, represantandose como NaN

In [132]:
def filtered_data():
    '''Filtering NO2 Data'''
    pattern = "(_8_)"
    filtered = df['PUNTO_MUESTREO'].str.extract(pattern)
    result = pd.concat([df, filtered], axis=1, sort=False)
    df_NO2=result[result[0]=='_8_']
    '''Filtering NO2 Data'''
    valid=['V01','V02','V03','V04','V05','V06', 'V07','V08','V09',
           'V10','V11', 'V12', 'V13', 'V14','V15','V16','V17','V18',
           'V19','V20','V21','V22','V23','V24']
    for v in valid:
        dataframe=df_NO2[df_NO2[v]=='V']
        
    return dataframe

He tomado la decisión de eliminar los registros no válidos (aquellas filas con un registro no valido:'N') tras comprobar que no eran demasiados(aprox 25). Por lo que he sopesado que es más beneficioso obtener solo valores válidos que mantenerlos para conseguir resultados más ajustados a la realidad. 
En el caso que hubiera demasiados datos no válidos que supusiera la eliminación de demasiados registros, se debería plantear la posibilidad de sustituir esos registros por otros aquellos que no influyera demasiado en los resultados, ya que evitaría el quedarnos con pocos.

Por otro lado en la función he decidido hacer una iteración de un array(valid) escrito a mano los campos, ya que al ser una prueba técnica he preferido no cargar de más código. En otra ocasión que hubiera más atributos y fuera un trabajo muy mecánico, se podría conseguir el array a través de otros métodos como las expresiones regulares que se han ido utilizando .

En la siguiente dos funciones se procede a revisar la existencia de valores Null/Nan.

In [84]:
def check_values_null():
    df = processing_data()
    df_null=df.isnull().values.ravel().sum()
    if df_null==0:
        print('There are not Null values')
    else:
        print(f'There are {df_null} null values')

def check_values_nan():
    df = processing_data()
    df_nan=df.isna().values.ravel().sum()
    if df_nan==0:
        print('There are not NaN values')
    else:
        print(f'There are {df_nan} NaN values')
        
check_values_nan()
check_values_null()

There are not NaN values
There are not Null values


### 1.4 Almacenamiento de datos
Para esta situación donde no hay un excesivo número de registros en el dataframe final, se procede a crear una función donde se almacene en un fichero .csv


In [137]:
dataframe=filtered_data()
file_name='CargaBD/NO2_data.csv'
dataframe.to_csv(file_name, sep=';')

### 1.5 APUNTES

* En una situación real, en la función de descompresión de los .zip, se debería de hacer una función de limpieza de los mismos ficheros tras finalizar el proceso de ETL, para mantener las carpetas limpias y en orden, a no ser que por otras razones no interese. 
* También en una situación donde se está en contacto directo con los demás departamentos, se debería de hacer una limpieza más exhaustiva. Cada proceso de tratamiento de datos, deben de ir acompañados de funciones donde revise que no haya pérdidas de datos durante el camino.Para este ejercicio práctico lo he realizado tras la carga de datos para mostrar la importancia de este proceso. Los siguientes apartados los he realizado de manera individual sin mostrar las funciones para centrarse en los demás procesos.
* He querido dividir el ejercicio en diferentes funciones con el fin de hacerlo más modular, además de intentar conseguir un código más limpio y claro de leer.
* A nivel personal no me gusta cargar de demasiados comentarios los códigos. Siempre intento de hacer un número razonable de comentarios, ya que si junto con el código se consigue un código limpio y facil de entender, es una buena muestra de conseguir buenas prácticas de codificación.
* En la carga de datos he hecho una simple carga de datos a .csv porque he considerado que no es un número excesivo de registro. En otras situaciones que se prefiera hacer carga a una base de datos, python permite hacer conexiones a base de datos y posteriormente carga de los datos.
